# 02 REST API | 01 Basic Chat

## Intro

Chat models take a series of messages as input, and return a model-generated message as output. The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content (the content of the message).

## Step 1:  Setup Parameters

In [ ]:
#r "nuget: DotNetEnv, 2.5.0"
#r "nuget: System.Text.Json, 7.0.3"
#r "nuget: Newtonsoft.Json, 13.0.1"
using DotNetEnv;

using System.Net;
using System.Net.Http;
using System.Text.Json.Nodes;
using System.Text.Json;

static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string apiBase = Environment.GetEnvironmentVariable("SKIT_AOAI_ENDPOINT"); 
string apiKey = Environment.GetEnvironmentVariable("SKIT_AOAI_APIKEY"); 
string deploymentName = Environment.GetEnvironmentVariable("SKIT_CHATCOMPLETION_DEPLOYMENTNAME"); 
string apiVersion = "2023-07-01-preview";

Expected output
```
Installed Packages
    DotNetEnv, 2.5.0
    Newtonsoft.Json, 13.0.1
    System.Text.Json, 7.0.3
```

## Step 2:  Create completions for chat messages with GPT models

The code cell is using an instance of `HttpClient` to call the REST API of the deployed Azure OpenAI instance.

In [ ]:

var requestPayload = new JsonObject
{
    { "messages", new JsonArray
        {
            new JsonObject
            {
                { "role", "system" },
                { "content", "You are an AI assistance who You extract intention from provided text. You always answer with intention:" }
                
            },
            new JsonObject
            {
                { "role", "user" },
                { "content", "I'm not receiving calls on my Samsung Galaxy S22. Can you help?" }
            }
        }
    },
    { "max_tokens", 200 },
    { "temperature", 0.7 },
    { "frequency_penalty", 0 },
    { "presence_penalty", 0 },
    { "top_p", 0.95 },
    { "stop", null }
};

string payload = JsonSerializer.Serialize(requestPayload, new JsonSerializerOptions
{
    WriteIndented = true // Optional: to make the JSON string more readable
});

        
string endpoint = $"{apiBase}openai/deployments/{deploymentName}/chat/completions?api-version={apiVersion}";

using (HttpClient httpClient = new HttpClient())
{
    httpClient.BaseAddress = new Uri(endpoint);
    httpClient.DefaultRequestHeaders.Add("api-key",apiKey);
    httpClient.DefaultRequestHeaders.Accept.Add(new System.Net.Http.Headers.MediaTypeWithQualityHeaderValue("application/json"));

    var stringContent = new StringContent(payload, Encoding.UTF8, "application/json");

    var response = await httpClient.PostAsync(endpoint, stringContent);

    if (response.IsSuccessStatusCode)
    {
        using (var responseStream = await response.Content.ReadAsStreamAsync())
        {
            // Parse the JSON response using JsonDocument
            using (var jsonDoc = await JsonDocument.ParseAsync(responseStream))
            {
                // Access the message content dynamically
                var root = jsonDoc.RootElement;
                var messageContent = root.GetProperty("choices")[0].GetProperty("message").GetProperty("content").GetString();

                // Output the message content
                Console.WriteLine("Output: " + messageContent);
            }
        }
    }
    else
    {
        Console.WriteLine($"Error: {response}");
    }
}

Expected output:

```
Output: Intention: Requesting technical assistance with phone call issue on Samsung Galaxy S22.
```

## Next Steps

Calling additional Open AI models [next notebook](02_OtherModels.ipynb).